# HDC Symbolic Query Demo — "Capital of France?"

**SC-NeuroCore v3.7 — Hyper-Dimensional Computing (HDC/VSA) Kernel**

This notebook demonstrates how SC-NeuroCore's SIMD-accelerated `BitStreamTensor` enables
symbolic reasoning via Hyper-Dimensional Computing on 10,000-bit binary vectors.

**Key operations:**
- **Bind** (`*` / XOR) — associates two concepts (self-inverse)
- **Bundle** (`+` / majority vote) — combines multiple items into a set
- **Permute** (cyclic rotation) — encodes position/sequence
- **Similarity** (1 − normalised Hamming distance) — measures relatedness

> CopyRight: (c) 1998-2026 Miroslav Sotek. All rights reserved.  
> License: GNU AFFERO GENERAL PUBLIC LICENSE v3 | Commercial Licensing Available  
> Contact: www.anulum.li   protoscience@anulum.li

In [ ]:
from sc_neurocore_engine import HDCVector

DIM = 10_000  # 10,000-bit hypervectors
print(f"SC-NeuroCore HDC Symbolic Query Demo (D={DIM})")

## Step 1: Create Atomic Symbols

Each concept (country, capital, role) gets a random 10,000-bit vector.
Random high-dimensional vectors are quasi-orthogonal: pairwise similarity ≈ 0.50.

In [ ]:
# Role vectors
role_country = HDCVector(DIM, seed=1000)
role_capital = HDCVector(DIM, seed=1001)

# Country atoms
france  = HDCVector(DIM, seed=2000)
germany = HDCVector(DIM, seed=2001)
japan   = HDCVector(DIM, seed=2002)

# Capital atoms
paris  = HDCVector(DIM, seed=3000)
berlin = HDCVector(DIM, seed=3001)
tokyo  = HDCVector(DIM, seed=3002)

atoms = {
    "France": france, "Germany": germany, "Japan": japan,
    "Paris": paris, "Berlin": berlin, "Tokyo": tokyo,
}

print(f"Pairwise similarity (should be ~0.50):")
print(f"  France vs Germany: {france.similarity(germany):.3f}")
print(f"  Paris vs Berlin:   {paris.similarity(berlin):.3f}")

## Step 2: Encode Records

Each country–capital pair is encoded as:

$$\text{record} = (\text{role\_country} \oplus \text{country}) + (\text{role\_capital} \oplus \text{capital})$$

where $\oplus$ is XOR-bind and $+$ is majority-vote bundle.

In [ ]:
rec_france  = HDCVector.bundle([role_country * france,  role_capital * paris])
rec_germany = HDCVector.bundle([role_country * germany, role_capital * berlin])
rec_japan   = HDCVector.bundle([role_country * japan,   role_capital * tokyo])

# Bundle all records into memory
memory = HDCVector.bundle([rec_france, rec_germany, rec_japan])
print("Records encoded and bundled into memory.")

## Step 3: Query — "Which countries are in memory?"

Unbind the capital role from memory to reveal country associations:

In [ ]:
probe = memory * role_capital

print("Query: 'Which countries are in memory?' (unbind role_capital)")
for name, atom in [("France", france), ("Germany", germany),
                   ("Japan", japan), ("Paris", paris)]:
    sim = probe.similarity(atom)
    print(f"  sim(probe, {name:>8s}) = {sim:.4f}")

## Step 4: Query — "Capital of France?"

Two-step unbinding:
1. Unbind France from memory: `hat = memory * france`
2. Unbind the capital role: `answer = hat * role_capital`

The result should be most similar to **Paris**.

In [ ]:
hat = memory * france
answer = hat * role_capital

print("Query: 'Capital of France?' (memory * France * role_capital)")
best_name, best_sim = "", -1.0
for name, atom in atoms.items():
    sim = answer.similarity(atom)
    if sim > best_sim:
        best_sim = sim
        best_name = name
    print(f"  sim(answer, {name:>8s}) = {sim:.4f}")

print(f"\nBest match: {best_name} (similarity {best_sim:.4f})")

## Step 5: Verify Bind-Inverse Property

XOR is self-inverse: $(a \oplus b) \oplus b = a$. This is the mathematical foundation
that makes unbinding work.

In [ ]:
a = HDCVector(DIM, seed=42)
b = HDCVector(DIM, seed=99)
recovered = (a * b) * b
sim = recovered.similarity(a)
print(f"Bind-inverse: sim(a, (a*b)*b) = {sim:.4f} (should be ~1.0)")

## Step 6: Permute for Sequence Encoding

Cyclic rotation produces quasi-orthogonal vectors, enabling position encoding:

$$\text{sim}(v, \pi^k(v)) \approx 0.50 \quad \text{for } k \neq 0$$

In [ ]:
v  = HDCVector(DIM, seed=42)
p1 = v.permute(1)
p2 = v.permute(2)
print(f"Permute orthogonality:")
print(f"  sim(v, permute(v,1)) = {v.similarity(p1):.4f}")
print(f"  sim(v, permute(v,2)) = {v.similarity(p2):.4f}")